# Psycopg2
This notebook is used to learn how to use the *psycopg2* library. It will create functions useful for the project

### Connection to DB

In [1]:
import psycopg2
from psycopg2 import Error

def connect_to_postgres_database(user, password, database, host="127.0.0.1", port="5432"):
    """
    Function connects to a database and returns the cursor object
    :param user: database username
    :param password: database password
    :param database: database name
    :param host: server location
    :param port: listening port
    :return: psycopg2 cursor object
    """
    try:
        con = psycopg2.connect(user=user,
                                      password=password,
                                      database=database,
                                      host=host,
                                      port=port)
        cursor = con.cursor()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    return con, cursor


In [87]:
# Connecting to DB
USER = "postgres"
PASSWORD = "Barca2011"
DATABASE = "golf_dashboard_db"

con, cursor = connect_to_postgres_database(USER, PASSWORD, DATABASE, host="127.0.0.1", port="5432")

### Table to Pandas DF

In [88]:
import pandas as pd

def table_to_dataframe(table):
    """
    Function returns a queried table in a Pandas DataFrame
    :param table: name of table in database
    :return: dataframe of table
    """
    cursor.execute("""SELECT * FROM {}""".format(table))
    tmp = cursor.fetchall()
    col_names = list()
    for elt in cursor.description:
        col_names.append(elt[0])
    df = pd.DataFrame(tmp, columns=col_names)
    return df

In [89]:
# Viewing table
course_df = table_to_dataframe("course")
course_df

,name,holes_18,city,slope,rating,id,par,country
0,Golf Costa Daurada,18,Tarragona,121.0,69.7,1,NaN,None
1,Loch Palm Golf Club,18,Kathu,129.0,68.4,3,NaN,None
2,Red Mountain Golf Club,18,Kathu,121.0,68.4,5,NaN,None
3,Aravell Golf Club,18,Aravell,139.0,74.0,14,71.0,None
4,A,18,B,12.0,12.0,38,12.0,C
5,Test Golf Course,18,Test,120.0,75.0,39,72.0,Test


### Inserting to course Table

In [122]:
def cursor_execute_tuple(command, data_tuple):
    """
    Function uses the cursor object to execute a command with a tuple pair. It commits and rollsback if error
    :param command: SQL query to be executed
    :param data_tuple: data pairing for SQL query variables
    :return:
    """
    try:
        cursor.execute(command, data_tuple)
        con.commit()
        print("Successfully executed the command")
    except:
        con.rollback()
        print("Could not successfully execute the command")
    return None

def insert_course_in_course_table(name, holes_18, city, slope, rating,  par, country):
    """
    Function inserts course information into the course table
    :param name: the name of the course (TEXT)
    :param holes_18: the number of holes the course has (INT)
    :param city: the city/location of the course (TEXT)
    :param slope: the slope of the course (FLOAT)
    :param rating: the rating of the course (FLOAT)
    :param par: the par of the course (INT)
    :param country: the country of the course (TEXT)
    :return:
    """
    insert_command = """INSERT INTO course
                  (name, holes_18, city, slope, rating,  par, country)
                  VALUES (%s, %s, %s, %s, %s, %s, %s);"""
    data_tuple = (name, holes_18, city, slope, rating, par, country)
    cursor_execute_tuple(insert_command, data_tuple)
    return None

In [29]:
insert_course_in_course_table("Test Golf Course", 18, "Test", 200, 75,  72, "Test")

Successfully executed the command


In [30]:
# View dataframe
course_df = table_to_dataframe("course")
course_df

,name,holes_18,city,slope,rating,id,par,country
0,Golf Costa Daurada,18,Tarragona,121.0,69.7,1,NaN,None
1,Loch Palm Golf Club,18,Kathu,129.0,68.4,3,NaN,None
2,Red Mountain Golf Club,18,Kathu,121.0,68.4,5,NaN,None
3,Aravell Golf Club,18,Aravell,139.0,74.0,14,71.0,None
4,A,18,B,12.0,12.0,38,12.0,C
5,Test Golf Course,18,Test,120.0,75.0,39,72.0,Test
6,Test Golf Course,18,Test,120.0,75.0,40,72.0,Test
7,Test Golf Course,18,Test,200.0,75.0,41,72.0,Test


### Insert to course_{} Table

In [116]:
course_id = 4
list_course_uid = [course_id]
tuple(list_course_uid)

(4,)

In [110]:
tuple([1, 3]) + tuple([1, 3])

(1, 3, 1, 3)

In [150]:
def make_data_tuple_9holes(table, course_feature, course_id):
    """
    Function creates the SQL command needed to insert features into a table - 9 holes
    :param table: name of table in database
    :param course_feature: list of features
    :return: insert SQL command, features as tuple
    """
    insert_command = """INSERT INTO {}
                  (course_id, hole1, hole2, hole3, hole4, hole5, hole6, hole7, hole8, hole9)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""".format(table)
    course_id_list = [course_id]
    course_id_tuple = tuple(course_id_list)
    feature_tuple = tuple(course_feature)
    data_tuple = tuple(course_id_tuple) + tuple(feature_tuple)
    return insert_command, data_tuple

def make_data_tuple_18holes(table, course_feature, course_id):
    """
    Function creates the SQL command needed to insert features into a table - 18 holes
    :param table: name of table in database
    :param course_feature: list of features
    :return: insert SQL command, features as tuple
    """
    insert_command = """INSERT INTO {}
                      (course_id, hole1, hole2, hole3, hole4, hole5, hole6, hole7, hole8, hole9, hole10, hole11, hole12, hole13, hole14, hole15, hole16, hole17, hole18)
                      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""".format(table)
    course_id_list = [course_id]
    course_id_tuple = tuple(course_id_list)
    feature_tuple = tuple(course_feature)
    data_tuple = tuple(course_id_tuple) + tuple(feature_tuple)
    return insert_command, data_tuple

def insert_score_card_feature_to_table(table, course_feature, course_id):
    """
    Function pipelines the process to insert 9/18 hole feature information (par, si, distance) into a database table
    :param table: name of table in database
    :param course_feature: list of features
    :return:
    """
    if len(course_feature) == 9:
        insert_command, data_tuple = make_data_tuple_9holes(table, course_feature, course_id)
    elif len(course_feature) == 18:
        insert_command, data_tuple = make_data_tuple_18holes(table, course_feature, course_id)
    cursor_execute_tuple(insert_command, data_tuple)
    return None

In [158]:
par_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9]
course_id = 3

insert_score_card_feature_to_table("course_par", par_list, course_id)

Successfully executed the command


### Get ID from Course Features

In [99]:
def get_id_from_course_name(course_name):
    """
    Function returns the id of the course in the course table based on the name
    :param course_name: name of course quiered
    :return: id of course quiered
    """
    insert_command = """SELECT id FROM course
                    WHERE name = %s;"""
    cursor.execute(insert_command, [course_name])
    returned_value = cursor.fetchall()
    id = returned_value[0][0]
    return id

In [120]:
get_id_from_course_name("Loch Palm Golf Club")

3

In [157]:
dataframe = pd.read_csv(r"C:\Users\Jaume\Downloads\golf_course_score_card_template (33).csv")
course_par = list(dataframe["Par"])
course_distance = list(dataframe["Distance"])
course_stroke_index = list(dataframe["Stroke Index"])
insert_score_card_feature_to_table("course_par", course_par, 3)
insert_score_card_feature_to_table("course_distance", course_distance, 3)
insert_score_card_feature_to_table("course_stroke_index", course_stroke_index, 3)

Successfully executed the command
Successfully executed the command
Successfully executed the command


In [133]:
insert_score_card_feature_to_table("course_par", course_par, 3)

Could not successfully execute the command


In [156]:
dataframe = pd.read_csv(r"C:\Users\Jaume\Downloads\golf_course_score_card_template (33).csv")
course_par = list(dataframe["Par"])
course_distance = list(dataframe["Distance"])
course_stroke_index = list(dataframe["Stroke Index"])
insert_score_card_feature_to_table("course_par", course_par, "232")
insert_score_card_feature_to_table("course_distance", course_distance, 43)
insert_score_card_feature_to_table("course_stroke_index", course_stroke_index, 43)

Could not successfully execute the command
Successfully executed the command
Successfully executed the command
